In [1]:
# Data: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv

In [1]:
import requests
import pandas as pd
import re
import numpy
from google.cloud import storage

In [2]:
URL = 'http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv'
FILEPATH = './data/uci_bogota_raw.csv'

In [3]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [4]:
ucis = pd.read_csv( './data/uci_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';', skiprows = 4, skipfooter = 3 )

<ipython-input-4-4892b66081bf>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  ucis = pd.read_csv( './data/uci_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';', skiprows = 4, skipfooter = 3 )


In [5]:
def convert_int64(column:str):
    temp=ucis[column]
    i=0
    for row in temp:
        data=str(row) 
        search=re.findall('^[1-9]\.\w*',data)
        if(len(search)==1):
            #Select first item in list of re
            str1=search[0]
            
            #To float to aproximate 3 digits
            str1=float(str1)
            str1=round(str1,3)
            #Cast as str and replace '.'
            str1=str(str1).replace('.','')
            if(len(str1)==3):
                str1=str1+'0'
                
            #Replace in the column
            temp[i]=str1
            temp[i]=temp[i].astype(numpy.int64)
            
        else:
            row=float(data)
            temp[i]=int(row)
            temp[i]=temp[i].astype(numpy.int64)
        i+=1
    
    ucis[column]=temp.astype(numpy.int64)

convert_int64('Camas UCI ocupadas Covid-19')
convert_int64('Camas UCI Disponibles COVID 19')

<ipython-input-5-f2d37cf11372>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[i]=int(row)
<ipython-input-5-f2d37cf11372>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[i]=temp[i].astype(numpy.int64)
<ipython-input-5-f2d37cf11372>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[i]=str1
<ipython-input-5-f2d37cf11372>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

In [6]:
ucis.shape

(126, 4)

In [7]:
ucis.dtypes

Fecha                             object
Camas UCI ocupadas Covid-19        int64
Camas UCI Disponibles COVID 19     int64
Ocupación UCI COVID 19            object
dtype: object

In [8]:
ucis[ 'Fecha' ] = pd.to_datetime( ucis[ 'Fecha' ], format = '%d/%m/%Y' )

In [9]:
ucis[ '% Ocupación' ] = ucis[ 'Camas UCI ocupadas Covid-19' ] / ucis[ 'Camas UCI Disponibles COVID 19' ]

In [10]:
ucis.dtypes

Fecha                             datetime64[ns]
Camas UCI ocupadas Covid-19                int64
Camas UCI Disponibles COVID 19             int64
Ocupación UCI COVID 19                    object
% Ocupación                              float64
dtype: object

In [11]:
del ucis[ 'Ocupación UCI COVID 19' ]

In [12]:
ucis.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [13]:
ucis.tail( 20 )

,Fecha,Camas UCI ocupadas Covid-19,Camas UCI Disponibles COVID 19,Ocupacion
106,2020-07-24,1345,1446,0.930152
107,2020-07-25,1324,1447,0.914997
108,2020-07-26,1356,1455,0.931959
109,2020-07-27,1349,1479,0.912103
110,2020-07-28,1358,1503,0.903526
111,2020-07-29,1356,1533,0.884540
112,2020-07-30,1384,1553,0.891178
113,2020-07-31,1398,1566,0.892720
114,2020-08-01,1426,1586,0.899117
115,2020-08-02,1412,1604,0.880299


In [14]:
#ucis.to_csv( './data/uci_bogota.csv', index = False )

In [15]:
#DataFrame to CSV
csv_data=ucis.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('uci_bogota.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')